In [9]:
import numpy as np
import pandas as pd
import pandas_ta as ta

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 8]
plt.style.use('fivethirtyeight')

from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split , GridSearchCV , cross_val_score , TimeSeriesSplit , RandomizedSearchCV

from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import classification_report , confusion_matrix , accuracy_score
from sklearn.metrics import roc_curve , plot_roc_curve , plot_confusion_matrix , auc


import cufflinks as cf
cf.set_config_file(offline=True)
#https://github.com/santosjorge/cufflinks

import seaborn as sns
import plotly.graph_objs as go


import yfinance as yf
from boruta import BorutaPy

In [10]:
import yfinance as yf
import numpy as np

list_of_stock = [
    'ADVANC.BK', 'AOT.BK', 'AWC.BK', 'BANPU.BK', 'BBL.BK', 'BDMS.BK',
    'BEM.BK', 'BGRIM.BK', 'BH.BK', 'BJC.BK', 'BTS.BK', 'CBG.BK', 
    'CENTEL.BK', 'COM7.BK', 'CPALL.BK', 'CPF.BK', 'CPN.BK', 'CRC.BK',
    'DTAC.BK', 'EA.BK', 'EGCO.BK', 'GLOBAL.BK', 'GPSC.BK', 'GULF.BK',
    'HMPRO.BK', 'INTUCH.BK', 'IRPC.BK', 'IVL.BK', 'KBANK.BK', 'KTB.BK',
    'KTC.BK', 'LH.BK', 'MINT.BK', 'MTC.BK', 'OR.BK', 'OSP.BK', 'PTT.BK',
    'PTTEP.BK', 'PTTGC.BK', 'RATCH.BK', 'SAWAD.BK', 'SCB.BK', 'SCC.BK',
    'SCGP.BK', 'TISCO.BK', 'TMB.BK', 'TOA.BK', 'TOP.BK', 'TRUE.BK', 'TTB.BK'
]

# Weights for each financial ratio
weights = {
    'forwardPE': 0.25,
    'priceToBook': 0.25,
    'debtToEquity': 0.2,
    'returnOnEquity': 0.2,
    'currentRatio': 0.1
}

# Initialize min and max values
min_values = {
    'forwardPE': float('inf'),
    'priceToBook': float('inf'),
    'debtToEquity': float('inf'),
    'returnOnEquity': float('-inf'),
    'currentRatio': float('inf')
}
max_values = {
    'forwardPE': float('-inf'),
    'priceToBook': float('-inf'),
    'debtToEquity': float('-inf'),
    'returnOnEquity': float('inf'),
    'currentRatio': float('-inf')
}

# Fetch financial data and determine min/max values
for stock_symbol in list_of_stock:
    stock = yf.Ticker(stock_symbol)
    try:
        info = stock.info
        if all(key in info for key in weights.keys()):
            min_values['forwardPE'] = min(min_values['forwardPE'], info['forwardPE'])
            max_values['forwardPE'] = max(max_values['forwardPE'], info['forwardPE'])
            min_values['priceToBook'] = min(min_values['priceToBook'], info['priceToBook'])
            max_values['priceToBook'] = max(max_values['priceToBook'], info['priceToBook'])
            min_values['debtToEquity'] = min(min_values['debtToEquity'], info['debtToEquity'])
            max_values['debtToEquity'] = max(max_values['debtToEquity'], info['debtToEquity'])
            min_values['returnOnEquity'] = max(min_values['returnOnEquity'], info['returnOnEquity'])
            max_values['returnOnEquity'] = min(max_values['returnOnEquity'], info['returnOnEquity'])
            min_values['currentRatio'] = min(min_values['currentRatio'], info['currentRatio'])
            max_values['currentRatio'] = max(max_values['currentRatio'], info['currentRatio'])
    except KeyError:
        continue

def normalize(value, min_value, max_value):
    """Normalize the value to a range of 0 to 1."""
    return (value - min_value) / (max_value - min_value) if max_value != min_value else 1

best_score = float('-inf')
best_stock = None

for stock_symbol in list_of_stock:
    stock = yf.Ticker(stock_symbol)
    try:
        info = stock.info
        # Check if all required ratios are present
        if all(key in info for key in weights.keys()):
            pe_ratio = info['forwardPE']
            pb_ratio = info['priceToBook']
            de_ratio = info['debtToEquity']
            roe = info['returnOnEquity']
            current_ratio = info['currentRatio']
            
            # Normalize ratios for comparison
            normalized_pe = normalize(pe_ratio, min_values['forwardPE'], max_values['forwardPE'])
            normalized_pb = normalize(pb_ratio, min_values['priceToBook'], max_values['priceToBook'])
            normalized_de = 1 - normalize(de_ratio, min_values['debtToEquity'], max_values['debtToEquity'])
            normalized_roe = normalize(roe, min_values['returnOnEquity'], max_values['returnOnEquity'])
            normalized_current_ratio = normalize(current_ratio, min_values['currentRatio'], max_values['currentRatio'])
            
            # Calculate composite score
            score = (normalized_pe * weights['forwardPE'] +
                     normalized_pb * weights['priceToBook'] +
                     normalized_de * weights['debtToEquity'] +
                     normalized_roe * weights['returnOnEquity'] +
                     normalized_current_ratio * weights['currentRatio'])
            
            if score > best_score:
                best_score = score
                best_stock = stock
    except KeyError:
        # Skip if any required financial ratio is not available
        continue

if best_stock:
    print(best_stock.info['symbol'], best_score)
    print(best_stock.info['forwardPE'])
else:
    print("No stock found with valid financial ratios.")


PTTEP.BK 0.6085772611839962
8.283379


In [11]:
# df1 = yf.download(best_stock.info['symbol'], start='2010-01-01' , end='2024-01-01')
df1 = yf.download('BH.bk', start='2010-01-01' , end='2024-01-01')

[*********************100%%**********************]  1 of 1 completed


In [12]:
df1

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,29.50,30.25,29.50,29.75,22.404995,231800
2010-01-05,30.25,30.25,29.75,29.75,22.404995,524400
2010-01-06,30.50,30.50,29.75,30.00,22.593275,1105700
2010-01-07,30.25,30.50,30.00,30.50,22.969826,245700
2010-01-08,30.25,30.75,30.00,30.00,22.593275,293100
...,...,...,...,...,...,...
2023-12-22,217.00,218.00,214.00,216.00,213.015793,2276200
2023-12-25,216.00,217.00,215.00,216.00,213.015793,447900
2023-12-26,216.00,223.00,216.00,222.00,218.932907,3407800


In [13]:
df = df1.copy()
df['Forward Returns'] = np.log(df['Adj Close']).diff().shift(-1)
df

,Open,High,Low,Close,Adj Close,Volume,Forward Returns
Date,,,,,,,
2010-01-04,29.50,30.25,29.50,29.75,22.404995,231800,0.000000
2010-01-05,30.25,30.25,29.75,29.75,22.404995,524400,0.008368
2010-01-06,30.50,30.50,29.75,30.00,22.593275,1105700,0.016529
2010-01-07,30.25,30.50,30.00,30.50,22.969826,245700,-0.016529
2010-01-08,30.25,30.75,30.00,30.00,22.593275,293100,0.008299
...,...,...,...,...,...,...,...
2023-12-22,217.00,218.00,214.00,216.00,213.015793,2276200,0.000000
2023-12-25,216.00,217.00,215.00,216.00,213.015793,447900,0.027399
2023-12-26,216.00,223.00,216.00,222.00,218.932907,3407800,-0.009050


In [14]:
df.dropna(inplace=True)
df.isnull().sum()

Open               0
High               0
Low                0
Close              0
Adj Close          0
Volume             0
Forward Returns    0
dtype: int64

Features

In [15]:
#features
#1. Moving Average 20 days cross 200 days
df['MA20'] = df['Adj Close'].rolling(window=20).mean()
df['MA200'] = df['Adj Close'].rolling(window=200).mean()
# if MA20 cross MA200 then 1 else 0
df['MA_Signal'] = np.where(df['MA20'] > df['MA200'], 1, 0)

#2. ohlc4
# df['OHLC4'] = (df['Open'] + df['High'] + df['Low'] + df['Close']) / 4

#3. RSI
df['RSI'] = ta.rsi(df['Adj Close'], length=14)
df['RSI_Signal'] = np.where(df['RSI'] > 70, 1, 0)


# #5. Bullish divergence
# df['Bullish'] = np.where((df['Adj Close'] < df['Adj Close'].shift(1)) & (df['RSI'] > df['RSI'].shift(1)), 1, 0)

# #6. Bearish divergence
# df['Bearish'] = np.where((df['Adj Close'] > df['Adj Close'].shift(1)) & (df['RSI'] < df['RSI'].shift(1)), 1, 0)


# 7. MACD
macd = ta.macd(df['Adj Close'], fast=12, slow=26, signal=9)
df['MACD'] = macd['MACD_12_26_9']
df['MACD_Signal'] = macd['MACDs_12_26_9']
df['MACD_Hist'] = macd['MACDh_12_26_9']

#if MACD cross MACD Signal then 1 else 0
df['MACD_Signal'] = np.where(df['MACD'] > df['MACD_Signal'], 1, 0)

#8. Stochastic Oscillator
stoch = ta.stoch(high=df['High'], low=df['Low'], close=df['Close'], length=14)
df['%K'] = stoch['STOCHk_14_3_3']
df['%D'] = stoch['STOCHd_14_3_3']

#if %K cross %D then 1 else 0
df['Stoch_Signal'] = np.where(df['%K'] > df['%D'], 1, 0)

#9. Bollinger Bands
bbands = ta.bbands(df['Adj Close'], length=20)
df['BB_High'] = bbands['BBU_20_2.0']
df['BB_Low'] = bbands['BBL_20_2.0']



df.dropna(inplace=True)

In [17]:
features = ['RSI','%K','%D','MACD']
# features = ['RSI_Signal','MACD','MA_Signal']
X = df[features].values
y = np.where(df['Forward Returns'] >= 0, 1, 0)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(2568, 4) (642, 4) (2568,) (642,)
